In [1]:
import os, sys
import pandas as pd
import numpy as np
import sklearn
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import matplotlib as mpl
import matplotlib.pyplot as plt

import tensorflow as tf
import sgml, sgnn, sgutil

print(sys.version)
for i in [pd, np, sklearn, lgb, xgb, cb, mpl, tf]:
    try:
        print(i.__name__, i.__version__)
    except:
        pass

2024-11-06 01:03:22.480074: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730855002.565877    4163 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730855002.590487    4163 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 01:03:22.796585: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


3.12.6 (main, Sep 30 2024, 02:19:13) [GCC 9.4.0]
pandas 2.2.3
numpy 1.26.4
sklearn 1.5.2
xgboost 2.1.2
catboost 1.2.5
matplotlib 3.8.4
tensorflow 2.18.0


# 데이터 불러오기

In [2]:
data_path = 'data'
result_path = 'result'
img_path = 'img'

if not os.path.isdir(data_path):
    os.mkdir(data_path)

files = {
    k: os.path.join(p, f)
    for k, p, f in [
        ('train', data_path, 'train.csv'),
        ('test', data_path, 'test.csv'),
        ('org_train', data_path, 'train_org.csv'),
        ('org_test', data_path, 'test_org.csv'),
        ('train_pkl', data_path, 'train.pkl'),
        ('org_pkl', data_path, 'org.pkl'),
        ('test_pkl', data_path, 'test.pkl'),
        ('var_pkl', data_path, 'var.pkl'),
        ('model_pkl', result_path, 'stk_s4_ep7.pkl'),
        ('model1_pkl', result_path, 'stk1_s4_ep7.pkl'),
        ('model2_pkl', result_path, 'stk2_s4_ep7.pkl')
    ]
}
sc = sgutil.SGCache(img_path, result_path)

In [3]:
df_test = pd.read_pickle(files['test_pkl'])
df_var = pd.read_pickle(files['var_pkl'])

In [4]:
df_var

,var_type,Description,min,max,na,count,n_unique,f32,i32,i16,i8,dtype,src
Age,(continous),Age of the Customer.,20.0,85.0,0.0,19682810.0,66.00,True,True,True,True,Int8,NaN
Gender,(dichotomous),Gender of the Customer.,NaN,NaN,0.0,19682810.0,2.00,True,True,True,True,Categorical,NaN
Driving_License,(dichotomous),"0 for customer not having DL, 1 for customer h...",0.0,1.0,0.0,19682810.0,2.00,True,True,True,True,Int8,NaN
Region_Code,(nominal),Unique code for the region of the customer.,0.0,52.0,0.0,19682810.0,53.25,True,True,True,True,Categorical,NaN
Previously_Insured,(dichotomous),"0 for customer not having vehicle insurance, 1...",0.0,1.0,0.0,19682810.0,2.00,True,True,True,True,Categorical,NaN
Vehicle_Age,(nominal),Age of the vehicle.,NaN,NaN,0.0,19682810.0,3.00,True,True,True,True,Categorical,NaN
Vehicle_Damage,(dichotomous),Customer got his/her vehicle damaged in the pa...,NaN,NaN,0.0,19682810.0,2.00,True,True,True,True,Categorical,NaN
Annual_Premium,(continous),The amount customer needs to pay as premium in...,2630.0,540165.0,0.0,19682810.0,46734.25,True,True,False,False,Float32,NaN
Policy_Sales_Channel,(nominal),Anonymized Code for the channel of outreaching...,1.0,163.0,0.0,19682810.0,150.75,True,True,True,False,Categorical,NaN
Vintage,(continous),"Number of Days, Customer has been associated w...",10.0,299.0,0.0,19682810.0,290.00,True,True,True,False,Int16,NaN


In [5]:
target = 'Response'
X_cat = df_var.loc[df_var['dtype'] == 'Categorical'].index.tolist()
X_bool = df_var.loc[(df_var['n_unique'] == 2) & (~df_var['dtype'].isin(X_cat))].index.tolist() + ['is_Annual_Premium_mode']
X_bool = [i for i in X_bool if i != target and i not in X_cat]
X_num = df_var.loc[~df_var.index.isin(X_cat + X_bool + ['id', 'Response (Dependent Feature)'])].index.tolist()
X_all = df_test.columns.tolist()
print("Target:", target)
print("Categorical:", np.array(X_cat))
print("Boolean:", np.array(X_bool))
print("Number:", np.array(X_num))

Target: Response
Categorical: ['Gender' 'Region_Code' 'Previously_Insured' 'Vehicle_Age'
 'Vehicle_Damage' 'Policy_Sales_Channel' 'Annual_Premium_c' 'Age_c'
 'Vintage_c' 'Region_Code_S' 'Policy_Sales_Channel_S' 'Annual_Premium_S'
 'VA' 'VAge' 'PA' 'VV' 'PAge' 'VP' 'PVc' 'PP' 'RV' 'V2V' 'RP' 'PV2' 'PV'
 'GV2' 'GP' 'AA' 'AV' 'VPAV' 'VPPV' 'VPPA' 'VPVGAP' 'VPVGAV' 'VPVGAR'
 'VPRV' 'VPVA' 'VPPAc' 'VPRAc']
Boolean: ['Driving_License' 'is_Annual_Premium_mode' 'is_Annual_Premium_mode']
Number: ['Age' 'Annual_Premium' 'Vintage']


In [6]:
df_test

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,...,VPAV,VPPV,VPPA,VPVGAP,VPVGAV,VPVGAR,VPRV,VPVA,VPPAc,VPRAc
id,,,,,,,,,,,,,,,,,,,,,
11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228,...,Yes_1_20_142,Yes_0_43.0_34,No_0_124.0_36,No_0_< 1 Year_Female_61_32.0,Yes_1_< 1 Year_Female_22_222,No_0_1-2 Year_Female_34_6.0,Yes_0_51.0_136,No_0_< 1 Year_2630.0,No_0_124.0_16278.0,No_0_35.0_16353.0
11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123,...,No_1_57_81,No_1_90.0_79,No_1_119.0_75,No_0_< 1 Year_Female_24_158.0,No_1_1-2 Year_Male_45_157,No_0_< 1 Year_Female_61_38.0,No_1_9.0_191,No_1_> 2 Years_41510.0,No_1_50.0_38872.0,No_1_20.0_33542.0
11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271,...,Yes_1_59_293,Yes_1_90.0_131,No_1_119.0_79,No_0_< 1 Year_Female_34_87.0,Yes_1_1-2 Year_Male_61_269,Yes_0_< 1 Year_Male_83_38.0,Yes_1_27.0_77,No_0_< 1 Year_15246.0,No_0_124.0_15246.0,No_0_35.0_17806.0
11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115,...,No_1_49_190,No_1_21.0_250,No_0_135.0_34,No_0_< 1 Year_Female_41_163.0,No_1_1-2 Year_Female_65_210,No_0_1-2 Year_Female_35_49.0,No_1_7.0_252,No_0_1-2 Year_27181.0,No_0_159.0_29717.0,No_0_41.0_47907.0
11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148,...,No_1_80_26,No_1_57.0_188,No_1_101.0_82,No_0_< 1 Year_Female_25_124.0,No_1_> 2 Years_Male_50_122,Yes_0_> 2 Years_Male_85_7.0,No_1_32.0_136,No_1_1-2 Year_36335.0,No_1_128.0_25179.0,No_1_24.0_31713.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19174659,Male,57,1,28.0,0,1-2 Year,Yes,51661.0,124.0,109,...,No_1_44_195,No_1_59.0_145,Yes_0_25.0_29,No_0_< 1 Year_Female_25_28.0,No_1_1-2 Year_Female_35_261,No_0_< 1 Year_Female_66_47.0,No_1_26.0_123,Yes_1_< 1 Year_39797.0,Yes_1_18.0_25159.0,Yes_1_45.0_31315.0
19174660,Male,28,1,50.0,1,< 1 Year,No,25651.0,152.0,184,...,Yes_0_46_160,Yes_0_22.0_194,No_0_129.0_46,No_0_< 1 Year_Female_42_25.0,Yes_0_1-2 Year_Female_47_42,No_0_> 2 Years_Male_75_11.0,Yes_0_25.0_122,No_0_1-2 Year_40953.0,No_0_131.0_27898.0,No_0_2.0_37310.0
19174661,Male,47,1,33.0,1,1-2 Year,No,2630.0,138.0,63,...,No_0_68_192,No_0_105.0_40,No_1_70.0_20,Yes_0_< 1 Year_Male_52_163.0,No_0_> 2 Years_Female_47_238,No_1_> 2 Years_Male_69_35.0,No_0_0.0_42,No_0_< 1 Year_15297.0,No_0_124.0_19186.0,No_0_35.0_17690.0


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score

cv = StratifiedKFold(5, shuffle=True, random_state=123)
ss = StratifiedShuffleSplit(1, train_size=0.8, random_state=123)

def predict(m, df_valid, X):
    return pd.Series(m.predict_proba(df_valid[X])[:, 1], index=df_valid.index)

def eval_metric(y_true, prds):
    return roc_auc_score(y_true[target].sort_index(), prds.sort_index())

def print_metrics(m):
    if len(m['train_metrics']) > 0:
        print("Valid. Score: {:.5f}±{:.5f}, Train Score: {:.5f}±{:.5f}".format(
            np.mean(m['valid_metrics']), np.std(m['valid_metrics']),
            np.mean(m['train_metrics']), np.std(m['train_metrics']))
        )
    else:
        print("Valid. Score: {:.5f}±{:.5f}".format(
            np.mean(m['valid_metrics']), np.std(m['valid_metrics'])
        ))

In [8]:
if os.path.isfile(files['model_pkl']):
    stk = sgml.SGStacking.load_model(files['model_pkl'])
else:
    stk = sgml.SGStacking(pd.read_pickle(files['train_pkl']), target, cv, predict, eval_metric, greater_better=True, return_train_scores=False)

# Logistic Regression

In [9]:
X_tgt = ['VPVA', 'VAge', 'PA', 'VV', 'PAge', 'VP', 'PVc', 'PP', 'RV', 'RP',  'VPPV', 'VPPA', 'VPVGAP', 'VPVGAV', 'VPVGAR', 'VPRV', 'VPAV', 'AA', 'AV', 'VPPAc', 'VPRAc']
X_ohe = ['Driving_License']
X_lr = [i for i in X_all if not i.endswith('_S')]
ct = ColumnTransformer([
    ('tgt', TargetEncoder(), X_tgt), 
    ('ohe', OneHotEncoder(drop='first'), X_ohe)
])
m, train_result = stk.eval_model('lr', LogisticRegression, {'C':100}, X_lr, ct, progress_callback = sgml.ProgressCallBack())
print_metrics(m)

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

Valid. Score: 0.88870±0.00022


In [10]:
if not 'lr' in stk.get_selected_model():
    m = stk.select_model('lr')
    stk.save_model(files['model_pkl'])

# LightGBM

## LGBM1

In [11]:
X_lgb_tgt = ['VPVA', 'VAge', 'PA', 'VV', 'PAge', 'VP', 'PVc', 'PP', 'RV', 'RP',  'VPPV', 'VPPA', 'VPVGAP', 'VPVGAV', 'VPVGAR', 'VPRV', 'VPAV', 'AA', 'AV', 'VPPAc', 'VPRAc']
X_lgb_cat = ['V2V', 'PV2', 'PV', 'GV2', 'GP', 'Driving_License', 'Vehicle_Age', 'Gender', 'Region_Code', 'Policy_Sales_Channel', 'Previously_Insured', 'Vehicle_Damage', 'Age_c', 'Vintage_c']
X_lgb_num = ['Annual_Premium', 'Age']
X_lgb = X_lgb_tgt + X_lgb_cat + X_lgb_num
result = stk.eval_model('lgbm1', lgb.LGBMClassifier, 
    {'verbose': -1, 'num_leaves': 255, 'n_estimators': 3000, 'learning_rate': 0.01, 'colsample_bytree': 0.5, 'min_child_samples': 16}, 
    X_lgb, 
    preprocessor=ColumnTransformer([
        ('tgt', TargetEncoder(), X_lgb_tgt),
        ('pt', 'passthrough', X_lgb_cat + X_lgb_num)
    ]).set_output(transform="pandas"), 
    result_proc = sgml.lgb_learning_result, 
    train_params={
        'fit_params': {
            'categorical_feature': ['pt__' + i for i in X_lgb_cat], 'eval_metric': 'auc', 
            'callbacks': [sgml.LGBMFitProgressbar()]
        },
        'valid_config_proc': sgml.gb_valid_config
    }, progress_callback = sgml.ProgressCallBack()
)

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

In [12]:
if not 'lgbm1' in stk.get_selected_model():
    m = stk.select_model('lgbm1')
    stk.save_model(files['model_pkl'])

Round:   0%|          | 0/3000 [00:00<?, ?it/s]

# CatBoost

In [13]:
import catboost as cb
import dproc

In [ ]:
X_cat = ['Gender', 'Region_Code_S', 'Vehicle_Age', 'Policy_Sales_Channel_S', 'Driving_License', 'Previously_Insured', 'Vehicle_Damage', 'Annual_Premium_S', 'Vintage_c', 'Age_c']
X_num = ['Age', 'Annual_Premium']
X_cb = X_cat + X_num
result = stk.eval_model('cb1', cb.CatBoostClassifier, 
    {
        'n_estimators': 1000, 'learning_rate': 0.1, 'max_depth': 9, 'task_type': 'GPU', 
        'eval_metric': 'AUC', 'cat_features': X_cat, 'verbose' : False
    }, 
    X_cb, 
    result_proc = sgml.cb_learning_result, 
    train_params = {
        'fit_params': {'verbose': 1000}#, 'callbacks': [sgml.CatBoostFitProgressbar(n_estimators=3000)]},
    }, progress_callback = sgml.ProgressCallBack()
)

In [ ]:
stk.get_model_results('cb1')

In [ ]:
if not 'cb1' in stk.get_selected_model():
    m = stk.select_model('cb1')
    stk.save_model(files['model_pkl'])

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 2.27s	remaining: 37m 42s
999:	total: 32m 50s	remaining: 0us


In [ ]:
from sklearn.linear_model import LogisticRegression

stk.eval_meta_model(LogisticRegression, {}, ['lgbm1', 'lr'])

# Neural Network

속성간의 상호작용이 복잡하게 들어가 있어 이를 모델링하기 위한 교차망 (Cross Network) 구조를 사용했습니다.

In [ ]:
class ResponseModel(tf.keras.Model):
    def __init__(self, emb_cols, cont_cols, f_size, left_layers, right_layers, top_layers):
        super().__init__()
        if emb_cols != None:
            self.emb_layers = {
                v: tf.keras.layers.Embedding(c, s, name=v) for v, c, s in emb_cols
            }
            self.emb_cc = tf.keras.layers.Concatenate(axis=-1)
            if left_layers != None:
                self.left_model = tf.keras.models.Sequential([
                    tf.keras.layers.Dense(**params) for params in left_layers
                ])
                self.right_model = tf.keras.models.Sequential([
                    tf.keras.layers.Dense(**params) for params in right_layers
                ])
                self.mul = tf.keras.layers.Multiply()
                self.left_slice_layer_1 = tf.keras.layers.Lambda(lambda x: x[:, :f_size])
                self.left_slice_layer_2 = tf.keras.layers.Lambda(lambda x: x[:, f_size:])
                self.right_slice_layer_1 = tf.keras.layers.Lambda(lambda x: x[:, :f_size])
                self.right_slice_layer_2 = tf.keras.layers.Lambda(lambda x: x[:, f_size:])
            else:
                self.left_model = None
        else:
            self.emb_layers = None
        self.top_cc = tf.keras.layers.Concatenate(axis=-1)
        self.top_model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(**params) for params in top_layers
        ])
        self.f_size = f_size
        self.cont_cols = cont_cols
        self.sigmoid = tf.keras.layers.Activation('sigmoid')

    def __call__(self, X):
        if self.emb_layers != None:
            X_emb = self.emb_cc([tf.squeeze(v(X[k]), axis=-2) for k, v in self.emb_layers.items()])
            if self.left_model !=None:
                X_left = self.left_model(X_emb)
                X_right = self.right_model(X_emb)
                X_left_1 = self.left_slice_layer_1(X_left)
                X_left_2 = self.left_slice_layer_2(X_left)
                
                X_right_1 = self.right_slice_layer_1(X_right)
                X_right_2 = self.right_slice_layer_2(X_right)
                X_mul = self.mul([X_left_1, X_right_1])
                cc_list = [X_left_2, X_right_2, X_mul]
            else:
                cc_list = [X_emb]
        else:
            cc_list = list()
        if self.cont_cols != None:
            cc_list.append(X[self.cont_cols])
        if len(cc_list) == 1:
            return self.sigmoid(
                self.top_model(cc_list[0])
            )
        else:
            return self.sigmoid(
                self.top_model(self.top_cc(cc_list))
            )

def to_tf_dataset(X, Y=None, sample_weights=None, cont=[], cat=[]):
    d = {}
    if len(cont) > 0:
        d['Continous'] =  X[cont].values
    for i in cat:
        d[i] = np.expand_dims(X[i].cat.codes, axis=-1)
    if Y is None:
        return tf.data.Dataset.from_tensor_slices(d)
    else:
        if type(Y) == pd.Series:
            if sample_weights is None:
                return tf.data.Dataset.from_tensor_slices((d, Y.values))
            else:
                return tf.data.Dataset.from_tensor_slices((d, Y.values, sample_weights.values))
        else:
            if sample_weights is None:
                return tf.data.Dataset.from_tensor_slices((d, Y))
            else:
                return tf.data.Dataset.from_tensor_slices((d, Y, sample_weights))

def nn_cat_param(df, name, size):
    return 'pt__' + name, len(df[name].cat.categories), size

def nn_cat_param2(df, name, size):
    return name, len(df[name].cat.categories), size

# XGBooost

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder, TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [ ]:
X_cat = ['Gender', 'Region_Code', 'Vehicle_Age', 'Policy_Sales_Channel', 'Driving_License', 'Previously_Insured', 'Vehicle_Damage', 'Vintage', 'Annual_Premium_c']
X_num = ['Age']
X_xgb = X_cat + X_num
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), X_cat),
    ('pt', 'passthrough', X_num)
])
stk.eval_model_cv(ss, xgb.XGBClassifier, 
    {'n_estimators': 4000, 'learning_rate': 0.05, 'max_depth': 9, 'tree_method': 'hist', 'device': 'cuda', 'eval_metric': 'auc'}, 
    X_xgb, 
    preprocessor=ct,
    result_proc = sgml.cb_learning_result, 
    train_params = {
        'valid_splitter': lambda x: train_test_split(x, train_size=0.9, stratify=x[target], random_state=123),
        'valid_config_proc': sgml.gb_valid_config,
        'fit_params': {'verbose': 100}
    }
)